In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact
import ipywidgets as widgets
from scipy.optimize import curve_fit

def absorbance_over_wavelength(df):
    x = df.iloc[1:, 0].values
    y_columns = df.columns[1:]
    fig, graph = plt.subplots(figsize=(20, 6))
    for col in y_columns:
        y = df[col].iloc[1:].values
        graph.plot(x, y, color="blue", alpha=0.1)
    graph.set_ylim(0, 4)
    graph.set_xlabel("Wavelength")
    graph.set_ylabel("Absorbance")
    graph.set_title("Absorbance over wavelength")
    plt.show()
    return 0

In [2]:
def lin_function(t, a, k):
    return a + k * t

def lin_function_fixed(t, k):
    return t * k

def lin_function_fixed_b(t, k, b_fixed):
    return t * k + b_fixed

def project1(col_index, mini, maxi, min_val, max_val, colors, palette, function, fixed_ord, df):
    fig, axes = plt.subplots(2, 2, figsize=(20, 10))
    graph1 = axes[0, 0]
    graph2 = axes[0, 1]
    graph3 = axes[1, 0]
    graph4 = axes[1, 1]
    #graph 1
    x = df.iloc[0:, 0].astype(str).str.replace(',', '.').astype(float).values
    y_columns = df.columns[2:]
    for col in y_columns:
        y = df[col].iloc[0:].astype(str).str.replace(',', '.').astype(float).values
        graph1.plot(x, y, color="blue", alpha=0.3)
    graph1.axvline(x[col_index], color="red")
    graph1.set_xlabel("Wavelength (nm)", fontsize=14)
    graph1.set_ylabel("Absorbance", fontsize=14)
    graph1.set_title("Absorbance over wavelength", fontsize=15)
    graph1.tick_params(axis='both', which='major', labelsize=13)
    graph1.figure.tight_layout(pad = 6.0)
    #graph 2
    concentration = df.iloc[0:, 1].dropna().astype(str).str.replace(',', '.').astype(float).values
    wavelength = df.iloc[0:, 0].dropna().astype(str).str.replace(',', '.').astype(float).values
    absorbance = []
    for i in range(0, len(df)):
        row = []
        for col in df.columns[2:]:
            val = str(df.iloc[i][col]).replace(',', '.')
            try:
                row.append(float(val))
            except ValueError:
                row.append(np.nan)
        absorbance.append(row)
    absorbance = np.clip(absorbance, 0, None)
    X, Y = np.meshgrid(wavelength, concentration)
    z = absorbance.T
    levels = np.linspace(min_val, max_val, colors)
    contour = graph2.contourf(X, Y, z, cmap=palette, levels = levels)
    cbar = fig.colorbar(contour, label="Absorbance")
    cbar.ax.yaxis.label.set_size(14)
    cbar.ax.tick_params(labelsize=13)
    graph2.axvline(wavelength[col_index], color="red")
    graph2.set_xlabel("Wavelength (nm)", fontsize=14)
    graph2.set_ylabel("Concentration (mol.L$^{-1}$)", fontsize=14)
    graph2.set_title("Concentration over wavelength", fontsize=15)
    graph2.tick_params(axis='both', which='major', labelsize=13)
    #graph 3
    #data
    y = df.iloc[col_index, 2:].dropna().astype(str).str.replace(',', '.').astype(float).values
    x = df.iloc[0:, 1].dropna().astype(str).str.replace(',', '.').astype(float).values
    wavelength = float(df.iloc[col_index, 0])
    #fit
    if function == 'linear (px)':
        params, cov = curve_fit(lin_function_fixed, x[mini:maxi], y[mini:maxi], p0=[1])
    elif function == 'affine (px + ord)':
        params, cov = curve_fit(lin_function, x[mini:maxi], y[mini:maxi], p0=[1, 0.1])
    elif function == 'affine_fixed (px + fixed_ord)':
        b_fixed = fixed_ord
        params, cov = curve_fit(lambda t, k: lin_function_fixed_b(t, k, b_fixed), x[mini:maxi], y[mini:maxi], p0=[1])
    #smooth fitted curve
    x_fit = np.linspace(min(x[mini:maxi]), max(x[mini:maxi]), 200)
    if function == 'linear (px)':
        y_fit = lin_function_fixed(x_fit, *params)
    elif function == 'affine (px + ord)':
        y_fit = lin_function(x_fit, *params)
    elif function == 'affine_fixed (px + fixed_ord)':
        y_fit = lin_function_fixed_b(x_fit, *params, b_fixed)
    perr = np.sqrt(np.diag(cov))
    #r_square
    if function == 'linear (px)':
        residuals = y - lin_function_fixed(x, params[0])
    elif function == 'affine (px + ord)':
        residuals = y - lin_function(x, params[0], params[1])
    elif function == 'affine_fixed (px + fixed_ord)':
        residuals = y - lin_function_fixed_b(x, params[0], b_fixed)
    ss_res = np.sum(residuals ** 2)
    ss_tot = np.sum((y - np.mean(y)) ** 2)
    r_square = 1 - (ss_res / ss_tot)
    #plot
    graph3.axvspan(x[mini], x[maxi - 1], facecolor="lightgray", alpha=0.3)
    graph3.plot(x, y, "bo", label="Data")
    if function == 'linear (px)':
        graph3.plot(x_fit, y_fit, label=f"Fit: r\u00B2={r_square:.4f}, p={params[0]:.1f} \u00B1 {perr[0]:.1f}, ord = 0", color="red")
    elif function == 'affine (px + ord)':
        graph3.plot(x_fit, y_fit, label=f"Fit: r\u00B2={r_square:.4f}, p={params[1]:.1f} \u00B1 {perr[1]:.1f}, ord = {params[0]:.1f} \u00B1 {perr[0]:.1f}", color="red")
    elif function == 'affine_fixed (px + fixed_ord)':
        graph3.plot(x_fit, y_fit, label=f"Fit: r\u00B2={r_square:.4f}, p={params[0]:.1f} \u00B1 {perr[0]:.1f}, ord = {b_fixed}", color="red")
    graph3.set_xlabel("Concentration (mol.L$^{-1}$)", fontsize=14)
    graph3.set_ylabel("Absorbance", fontsize=14)
    graph3.set_title(f"Absorbance over Concentration at {wavelength:.0f} nm (p = slope in g·L$^{{-1}}$)", fontsize=15)
    graph3.legend(fontsize=14)
    graph3.tick_params(axis='both', which='major', labelsize=13)
    #graph 4
    erreur_abs = (y * 0.01) / np.sqrt(3)
    residuals_norm = residuals / erreur_abs
    graph4.scatter(concentration, residuals_norm)
    graph4.axhspan(-2, 2, facecolor="lightgray", alpha=0.3)
    graph4.set_xlabel("Concentration (mol.L$^{-1}$)", fontsize=14)
    graph4.set_ylabel("Normalised residuals", fontsize=14)
    graph4.tick_params(axis='both', which='major', labelsize=13)
    plt.show()
    return 0

In [3]:
data = pd.read_csv("Canevas_Projet1_Fluo.csv", sep = ';') # Change the filename to your actual file

def safe_project(col_index, window_vals, range_vals, colors, palette, function, fixed_ord):
    mini, maxi = window_vals
    if mini >= maxi:
        maxi = mini + 1
    min_val, max_val = range_vals
    if min_val >= max_val:
        max_val = min_val + 1
    project1(col_index, mini, maxi, min_val, max_val, colors, palette, function, fixed_ord, data)

wavelengths = data.iloc[0:, 0].values
options = [(str(val), idx) for idx, val in enumerate(wavelengths)]

spectrum_number = len(data.iloc[0, 2:])
absorbance = []
for i in range(0, len(data)):
    row = []
    for col in data.columns[2:]:
        val = str(data.iloc[i][col]).replace(',', '.')
        try:
            row.append(float(val))
        except ValueError:
            row.append(np.nan)
    absorbance.append(row)
max_value = max([max(row) for row in absorbance])
min_value = min([min(row) for row in absorbance])

col_index_slider = widgets.SelectionSlider(
    options=options,
    value=50,
    description="Wavelength",
    layout=widgets.Layout(width='800px'),
)

range_slider = widgets.IntRangeSlider(
    value=[0, int(max_value)],
    min=0,
    max=int(max_value),
    step=1,
    description='Range',
    layout=widgets.Layout(width = '800px'),
)

window_slider = widgets.IntRangeSlider(
    value=[0, spectrum_number],
    min=0,
    max=spectrum_number,
    step=1,
    description='Spectrum number',
    layout=widgets.Layout(width = '800px'),
)

colors_slider = widgets.IntSlider(
    value=128,
    min=8,
    max=256,
    description='Colors',
    layout=widgets.Layout(width = "800px"),
)

color_palette = widgets.ToggleButtons(
    options=['plasma','viridis','inferno','magma','cividis'],
    description='Palette',
    orientation = 'horizontal',
    layout=widgets.Layout(width='auto')
)

function_choice = widgets.ToggleButtons(
    options=['affine (px + ord)', 'linear (px)', 'affine_fixed (px + fixed_ord)'],
    description='Fonction',
    orientation = 'horizontal',
    layout=widgets.Layout(width='400px')
)

fixed_ord = widgets.FloatText(
    value=0.0,
    description='Fixed ord:',
    layout=widgets.Layout(width='200px')
)

ui = widgets.HBox([
    widgets.VBox([
        col_index_slider,
        window_slider,
        range_slider,
        colors_slider
    ]),
    widgets.VBox([
        color_palette,
        function_choice,
        fixed_ord
    ]),
])

out = widgets.interactive_output(
    safe_project,
    {"col_index": col_index_slider, "window_vals":window_slider, "range_vals":range_slider, "colors": colors_slider, "palette":color_palette, "function":function_choice, "fixed_ord": fixed_ord}
)

display(widgets.VBox([out, ui]))
